In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics deep-sort-realtime opencv-python-headless pandas kaggle

In [ ]:
from ultralytics import YOLO

In [ ]:
yaml_content = """
train: /kaggle/input/garbage-data/YOLO-Waste-Detection-1/YOLO-Waste-Detection-1/train/images
val: /kaggle/input/garbage-data/YOLO-Waste-Detection-1/YOLO-Waste-Detection-1/valid/images
test: /kaggle/input/garbage-data/YOLO-Waste-Detection-1/YOLO-Waste-Detection-1/test/images

nc: 5
names: ['Organic', 'Plastic', 'Paper', 'Metal', 'E-Waste']
"""

with open("/kaggle/working/data.yaml", "w") as f:
    f.write(yaml_content)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(
    data="/kaggle/working/data.yaml",
    epochs=25,
    imgsz=544,
    batch=12,
    device="0,1"   # ✅ use both GPUs
)

In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import matplotlib.pyplot as plt

# 🔹 Paths ggg
weights_path = "/kaggle/working/runs/detect/train/weights/best.pt"  # trained YOLO weights
video_path = "/kaggle/input/waste-input/waste_input.mp4"           # input video
output_path = "/kaggle/working/output_tracked.avi"                 # output video

# 🔹 Class names from your dataset
class_names = ['Organic', 'Plastic', 'Paper', 'Metal', 'E-Waste']

# 🔹 Load YOLO model
model = YOLO(weights_path)

# 🔹 Initialize DeepSORT
tracker = DeepSort(max_age=30, n_init=3, nms_max_overlap=1.0)

# 🔹 Open video and output writer
cap = cv2.VideoCapture(video_path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model.predict(frame, imgsz=640, conf=0.5, verbose=False)

    detections = []
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            detections.append([[x1, y1, x2-x1, y2-y1], conf, cls])

    # Update DeepSORT tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw boxes with class name + track ID
    for track in tracks:
        if not track.is_confirmed():
            continue
        x1, y1, x2, y2 = track.to_ltrb()
        track_id = track.track_id
        
        # Assign class name from detection
        cls_name = class_names[track.det_class] if hasattr(track, 'det_class') else 'Unknown'
        
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
        cv2.putText(frame, f"{cls_name} ID {track_id}", (int(x1), int(y1)-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()

print(f"✅ Tracking finished. Saved at: {output_path}")

# 🔹 Display a sample frame
cap = cv2.VideoCapture(output_path)
ret, frame = cap.read()
cap.release()
if ret:
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Sample Output Frame")
    plt.show()

# 🔹 Create download link in Kaggle
from IPython.display import FileLink
FileLink(output_path)